In [ ]:
#CWR calculation example 
import netCDF4 as nc
import numpy as np
import pandas as pd

# Load the NetCDF file
file_path = '/monthly_E.nc'
ds = nc.Dataset(file_path)

# Extract variables
lon = ds.variables['lon'][:]
lat = ds.variables['lat'][:]
time = ds.variables['time'][:]
E = ds.variables['E'][:]

# Estimated Kc values for maize, rice, and wheat for each growth stage
Kc_values = {
    'maize': {
        'initial': 0.35,
        'middle': 1.25,
        'maturity': 0.65
    },
    'rice': {
        'initial': 1.1,
        'middle': 1.25,
        'maturity': 0.95
    },
    'wheat': {
        'initial': 0.35,
        'middle': 1.15,
        'maturity': 0.35
    }
}
#estimated plating months obtained from differnt internet (FAO) etc sources 
planting_months = {
    'maize': {
        'United States of America': 4, 'Argentina': 10, 'India': 6, 'Brazil': 1, 'Australia': 10, 'France': 5,
        'Russian Federation': 5, 'Canada': 5, 'Ukraine': 5, 'Romania': 5, 'Thailand': 5, 'Germany': 5, 'Poland': 5,
        'Viet Nam': 5, 'Pakistan': 7, 'Kazakhstan': 5, 'Bulgaria': 5, 'Hungary': 5, 'South Africa': 10, 'Paraguay': 10,
        'Myanmar': 4, 'Italy': 5, 'Lithuania': 5, 'Czech Republic': 5, 'China': 5, 'Latvia': 5, 'Slovakia': 5,
        'Uruguay': 10, 'Belgium': 5, 'United Kingdom': 5, 'Serbia': 5, 'Austria': 5, 'Netherlands': 5, 'Türkiye': 5,
        'Spain': 5, 'Croatia': 5, 'Mexico': 5, 'Cambodia': 5, 'Moldova, Republic of': 5, 'Denmark': 5, 'Sweden': 5,
        'Greece': 5, 'Estonia': 5, 'Tanzania, United Republic of': 10, 'United Arab Emirates': 4, 'Iraq': 4,
        'Guyana': 10, 'Portugal': 5, 'Peru': 10, 'Zambia': 10, 'Slovenia': 5, 'Lao People\'s Democratic Republic': 5,
        'Chile': 10, 'Finland': 5, 'Uganda': 10, 'Taipei, Chinese': 5, 'Ireland': 5, 'Japan': 5, 'Singapore': 4,
        'Bolivia, Plurinational State of': 10, 'Free Zones': 4, 'Indonesia': 4, 'Malaysia': 4, 'Senegal': 10,
        'Oman': 4, 'Belarus': 5, 'Ecuador': 10, 'Venezuela, Bolivarian Republic of': 10, 'Suriname': 10,
        'Côte d\'Ivoire': 10, 'Jordan': 4, 'Korea, Republic of': 5, 'Macedonia, North': 5, 'Luxembourg': 5,
        'Bangladesh': 4, 'Syrian Arab Republic': 4, 'Sudan': 10, 'Hong Kong, China': 5, 'Egypt': 4, 'Kenya': 10,
        'Guatemala': 10, 'Malawi': 10, 'Sri Lanka': 10, 'Uzbekistan': 5, 'Philippines': 5, 'Mozambique': 10,
        'Costa Rica': 10, 'Bosnia and Herzegovina': 5, 'Honduras': 10, 'Palestine, State of': 4, 'Saudi Arabia': 4,
        'Ethiopia': 10, 'Kyrgyzstan': 5, 'El Salvador': 10, 'Switzerland': 5
    },
    'rice': {
        'United States of America': 5, 'Argentina': 12, 'India': 6, 'Brazil': 1, 'Australia': 10, 'France': 6,
        'Russian Federation': 5, 'Canada': 5, 'Ukraine': 5, 'Romania': 5, 'Thailand': 5, 'Germany': 6, 'Poland': 5,
        'Viet Nam': 5, 'Pakistan': 6, 'Kazakhstan': 5, 'Bulgaria': 5, 'Hungary': 5, 'South Africa': 10, 'Paraguay': 10,
        'Myanmar': 4, 'Italy': 5, 'Lithuania': 5, 'Czech Republic': 5, 'China': 5, 'Latvia': 5, 'Slovakia': 5,
        'Uruguay': 10, 'Belgium': 5, 'United Kingdom': 5, 'Serbia': 5, 'Austria': 5, 'Netherlands': 5, 'Türkiye': 5,
        'Spain': 5, 'Croatia': 5, 'Mexico': 5, 'Cambodia': 5, 'Moldova, Republic of': 5, 'Denmark': 5, 'Sweden': 5,
        'Greece': 5, 'Estonia': 5, 'Tanzania, United Republic of': 10, 'United Arab Emirates': 4, 'Iraq': 4,
        'Guyana': 10, 'Portugal': 5, 'Peru': 10, 'Zambia': 10, 'Slovenia': 5, 'Lao People\'s Democratic Republic': 5,
        'Chile': 10, 'Finland': 5, 'Uganda': 10, 'Taipei, Chinese': 5, 'Ireland': 5, 'Japan': 5, 'Singapore': 4,
        'Bolivia, Plurinational State of': 10, 'Free Zones': 4, 'Indonesia': 4, 'Malaysia': 4, 'Senegal': 10,
        'Oman': 4, 'Belarus': 5, 'Ecuador': 10, 'Venezuela, Bolivarian Republic of': 10, 'Suriname': 10,
        'Côte d\'Ivoire': 10, 'Jordan': 4, 'Korea, Republic of': 5, 'Macedonia, North': 5, 'Luxembourg': 5,
        'Bangladesh': 4, 'Syrian Arab Republic': 4, 'Sudan': 10, 'Hong Kong, China': 5, 'Egypt': 4, 'Kenya': 10,
        'Guatemala': 10, 'Malawi': 10, 'Sri Lanka': 10, 'Uzbekistan': 5, 'Philippines': 5, 'Mozambique': 10,
        'Costa Rica': 10, 'Bosnia and Herzegovina': 5, 'Honduras': 10, 'Palestine, State of': 4, 'Saudi Arabia': 4,
        'Ethiopia': 10, 'Kyrgyzstan': 5, 'El Salvador': 10, 'Switzerland': 5
    },
    'wheat': {
        'United States of America': 10, 'Argentina': 5, 'India': 11, 'Brazil': 5, 'Australia': 5, 'France': 10,
        'Russian Federation': 9, 'Canada': 10, 'Ukraine': 9, 'Romania': 9, 'Thailand': 11, 'Germany': 10, 'Poland': 10,
        'Viet Nam': 11, 'Pakistan': 11, 'Kazakhstan': 9, 'Bulgaria': 10, 'Hungary': 10, 'South Africa': 11, 'Paraguay': 5,
        'Myanmar': 11, 'Italy': 10, 'Lithuania': 10, 'Czech Republic': 10, 'China': 10, 'Latvia': 10, 'Slovakia': 10,
        'Uruguay': 5, 'Belgium': 10, 'United Kingdom': 10, 'Serbia': 10, 'Austria': 10, 'Netherlands': 10, 'Türkiye': 10,
        'Spain': 10, 'Croatia': 10, 'Mexico': 11, 'Cambodia': 11, 'Moldova, Republic of': 10, 'Denmark': 10, 'Sweden': 10,
        'Greece': 10, 'Estonia': 10, 'Tanzania, United Republic of': 11, 'United Arab Emirates': 11, 'Iraq': 11,
        'Guyana': 5, 'Portugal': 10, 'Peru': 11, 'Zambia': 11, 'Slovenia': 10, 'Lao People\'s Democratic Republic': 11,
        'Chile': 5, 'Finland': 10, 'Uganda': 11, 'Taipei, Chinese': 10, 'Ireland': 10, 'Japan': 10, 'Singapore': 11,
        'Bolivia, Plurinational State of': 5, 'Free Zones': 11, 'Indonesia': 11, 'Malaysia': 11, 'Senegal': 11,
        'Oman': 11, 'Belarus': 10, 'Ecuador': 5, 'Venezuela, Bolivarian Republic of': 5, 'Suriname': 5,
        'Côte d\'Ivoire': 11, 'Jordan': 11, 'Korea, Republic of': 10, 'Macedonia, North': 10, 'Luxembourg': 10,
        'Bangladesh': 4, 'Syrian Arab Republic': 4, 'Sudan': 10, 'Hong Kong, China': 5, 'Egypt': 4, 'Kenya': 10,
        'Guatemala': 10, 'Malawi': 10, 'Sri Lanka': 10, 'Uzbekistan': 5, 'Philippines': 5, 'Mozambique': 10,
        'Costa Rica': 10, 'Bosnia and Herzegovina': 5, 'Honduras': 10, 'Palestine, State of': 4, 'Saudi Arabia': 4,
        'Ethiopia': 10, 'Kyrgyzstan': 5, 'El Salvador': 10, 'Switzerland': 5
    }
}


# Convert time variable to datetime
time_units = ds.variables['time'].units
time_calendar = ds.variables['time'].calendar
times = nc.num2date(time, units=time_units, calendar=time_calendar)

# Function to get Kc for a given crop, country, and month based on planting month
def get_kc(crop, country, month):
    planting_month = planting_months[crop][country]
    growth_stage_months = (month - planting_month) % 12
    
    if 0 <= growth_stage_months < 3:
        return Kc_values[crop]['initial']
    elif 3 <= growth_stage_months < 6:
        return Kc_values[crop]['middle']
    else:
        return Kc_values[crop]['maturity']

# Function to calculate CWR for a given crop and country
def calculate_cwr(E, crop, country, times):
    CWR = np.zeros_like(E)
    for i, date in enumerate(times):
        month = date.month
        kc = get_kc(crop, country, month)
        CWR[i] = E[i] * kc
    return CWR

# List of all countries
countries = list(planting_months['maize'].keys())

# Initialize arrays to store CWR for each crop
CWR_maize = np.zeros_like(E)
CWR_rice = np.zeros_like(E)
CWR_wheat = np.zeros_like(E)

# Calculate CWR for each crop globally
for country in countries:
    CWR_maize_country = calculate_cwr(E, 'maize', country, times)
    CWR_rice_country = calculate_cwr(E, 'rice', country, times)
    CWR_wheat_country = calculate_cwr(E, 'wheat', country, times)
    
    CWR_maize += CWR_maize_country
    CWR_rice += CWR_rice_country
    CWR_wheat += CWR_wheat_country

# Average the CWR over all countries
CWR_maize /= len(countries)
CWR_rice /= len(countries)
CWR_wheat /= len(countries)

# Save results to new NetCDF files
def save_to_netcdf(output_file_path, CWR_data, crop_name):
    with nc.Dataset(output_file_path, 'w', format='NETCDF4') as dst:
        dst.createDimension('time', len(time))
        dst.createDimension('lat', len(lat))
        dst.createDimension('lon', len(lon))
        
        times_var = dst.createVariable('time', 'f4', ('time',))
        latitudes = dst.createVariable('lat', 'f4', ('lat',))
        longitudes = dst.createVariable('lon', 'f4', ('lon',))
        cwr = dst.createVariable(f'CWR_{crop_name}', 'f4', ('time', 'lat', 'lon'), zlib=True)
        
        times_var[:] = time
        latitudes[:] = lat
        longitudes[:] = lon
        cwr[:] = CWR_data
        
        cwr.units = 'mm/month'
        cwr.long_name = f'Crop Water Requirement for {crop_name}'

    print(f'Saved CWR data for {crop_name} to {output_file_path}')

# Save CWR data for maize, rice, and wheat
save_to_netcdf('CWR_maize_global.nc', CWR_maize, 'maize')
save_to_netcdf('CWR_rice_global.nc', CWR_rice, 'rice')
save_to_netcdf('CWR_wheat_global.nc', CWR_wheat, 'wheat')
